In [1]:
import os
import json
import pickle

In [2]:
base_path = "/home/seb/thesis/mitmdump_sample" 
user_ids = os.listdir("/home/seb/thesis/mitmdump_sample")

In [3]:
#select users who made over a 100 posts
relevant_users = []
for u_id in user_ids:
    if len(os.listdir(f"{base_path}/{u_id}/posts")) > 100:
        relevant_users.append(u_id)

In [4]:
#parse in post and comment data from directories
posts = list()
comments = list()
for user_id in relevant_users:
    for post_id in os.listdir(f"{base_path}/{user_id}/posts/"):
        with open(f"{base_path}/{user_id}/posts/{post_id}/post.json", "r") as f:
            post_dict = json.load(f)
        posts.append({"user_id": user_id,
                                "post_id": post_id,
                                "content": post_dict["content"],
                                "timestamp": post_dict["createdOn2"]})
        if "moodId" in post_dict:
            posts[-1]["mood_id"] = post_dict["moodId"]
        if "newMood" in post_dict:
            posts[-1]["mood_name"] = post_dict["newMood"]
        if len(os.listdir(f"{base_path}/{user_id}/posts/{post_id}")) > 1:
            for comment_id in os.listdir(f"{base_path}/{user_id}/posts/{post_id}"):
                if comment_id == "post.json":
                    continue
                elif "reply" in comment_id:
                    with open(f"{base_path}/{user_id}/posts/{post_id}/{comment_id}", "r") as f:
                        chain = json.load(f)
                    for comment in chain["replies"]:
                        comments.append({"post_id": post_id,
                                            "content": comment["content"],
                                            "timestamp": comment["createdOn"],
                                            "author": comment["createdBy"]["id"],
                                            "reply_to": [i["userId"] for i in comment["taggedUsers"]]})
                else:
                    with open(f"{base_path}/{user_id}/posts/{post_id}/{comment_id}", "r") as f:
                        comment = json.load(f)
                    comments.append({"post_id": post_id,
                                        "content": comment["content"],
                                        "timestamp": comment["createdOn"],
                                        "author": comment["createdBy"]["id"],
                                        "reply_to": ""})

In [7]:
#write post and comments to file
with open("posts.pickle", "wb") as f:
    pickle.dump(posts, f)
with open("comments.pickle", "wb") as f:
    pickle.dump(comments, f)